# XML FILES

**by Leyla Yiğit**

## Working with XML/HTML Objects

### Hurriyet  XML Query

### A real XML/HTML dataset

On the 2nd of December 2018, I scraped 994 realty listing pages of residences for sale in Mecidiyekoy, Sisli neighbourhood from www.hurriyetemlak.com

In [1]:
datadir=~/data
hemlak=$datadir/he_sisli

In [2]:
ls -l $hemlak | head

total 125848
-rw-r--r-- 1 jovyan jovyan 122385 Dec 15 13:02 10005103
-rw-r--r-- 1 jovyan jovyan 126480 Dec 15 13:02 1108042
-rw-r--r-- 1 jovyan jovyan 116865 Dec 15 13:02 12497729
-rw-r--r-- 1 jovyan jovyan 126636 Dec 15 13:02 12748522
-rw-r--r-- 1 jovyan jovyan 113609 Dec 15 13:02 14363743
-rw-r--r-- 1 jovyan jovyan 121665 Dec 15 13:02 1514239
-rw-r--r-- 1 jovyan jovyan 126724 Dec 15 13:02 15395342
-rw-r--r-- 1 jovyan jovyan 127307 Dec 15 13:02 16603754
-rw-r--r-- 1 jovyan jovyan 126016 Dec 15 13:02 16723985
ls: write error: Broken pipe


Viewing the data 

In [ ]:
find $hemlak -mindepth 1 | head -1 | xargs cat

An XML/HTMl parser can parse, pretty print, and traverse through the files.

Xidel is highly performant:

Another way to view an indented version of the XML/HTMl file might be:

In [ ]:
find $hemlak -mindepth 1 | head -1 | xargs -i xidel --input-format html --output-format html -e "/" {}

### Extract tables from XML/HTML

We will extract all the information to the left of the picture in a listing as a table using R

First parse the file as html:

The houses that avaible for loan.

In [ ]:
he_kredi=$(for i in $hemlak/*; do \
kredi=$(xidel --input-format html --output-format adhoc -e \
"//div[@class='realty-details realty-details-right clearfix']/ul[@class='clearfix']//span[text()='Krediye Uygunluk']/following-sibling::span/text()" \
$i 2> /dev/null);
if [[ -z $kredi ]]; then echo NA; else echo $kredi; fi;
done)


In [14]:
echo "$he_kredi" | column -s $'\t' -t

ERROR: Error in parse(text = x, srcfile = src): <text>:1:6: unexpected string constant
1: echo "$he_kredi"
         ^


In [7]:
he_prices=$(for i in $hemlak/*; do \
xidel --input-format html --output-format adhoc -e \
"//div[@class='realty-details realty-details-right clearfix']/ul[@class='clearfix']/li[@class='price-line clearfix']/span/text()" \
$i 2> /dev/null | \
tr -d "." | grep -Po "\d+";
done)

In [8]:
echo "$he_prices" | wc -l

994


In [9]:
echo "$he_prices" | head -10

690000
440000
490000
580000
645000
420000
160000
382000
360000
450000


In [10]:
selected_info=$(paste <((echo "$he_prices")) \
<(echo "$he_kredi") --delimiters "\t")

In [19]:
echo "$selected_info" | head -20

690000	Uygun
440000	Uygun
490000	Uygun
580000	Uygun
645000	Uygun
420000	Uygun
160000	Uygun değil
382000	Uygun
360000	Uygun
450000	Uygun değil
480000	Uygun
200000	Uygun değil
300000	Uygun değil
250000	Uygun değil
900000	Uygun
300000	Uygun değil
425000	Uygun
410000	Uygun
595000	Uygun
300000	Uygun değil


In [20]:
Actually there is no enough analysis to decide to hypothesis. It seems that, prices does not directly affected credit avaibility. 
The houses that does not have credit avaibility seems cheaper but, other variables should have been considered. Therefore, I want to analyze situationofbuilding and age also.

bash: Actually: command not found


: 127

In [38]:
he_situation=$(for i in $hemlak/*; do \
yapinindurumu=$(xidel --input-format html --output-format adhoc -e \
"//div[@class='realty-details realty-details-right clearfix']/ul[@class='clearfix']//span[text()='Yapının Durumu']/following-sibling::span/text()" \
$i 2> /dev/null);
if [[ -z $yapinindurumu ]]; then echo NA; else echo $yapinindurumu; fi;
done)

In [22]:
echo "$he_yapinindurumu" | head -10

İkinci El
İkinci El
Sıfır
İkinci El
İkinci El
NA
İkinci El
İkinci El
Sıfır
İkinci El


In [23]:
he_age=$(for i in $hemlak/*; do \
age=$(xidel --input-format html --output-format adhoc -e \
"//div[@class='realty-details realty-details-right clearfix']/ul[@class='clearfix']//span[text()='Bina Yaşı']/following-sibling::span/text()" \
$i 2> /dev/null);
if [[ -z $age ]]; then echo NA; else echo $age; fi;
done)


In [24]:
echo "$he_age" | head -10

20
24
3
38
20
25
20
10
0
15


In [39]:
selected_info=$(paste <(ls $hemlak) <(echo "$he_prices") <(echo "$he_situation") \
<(echo "$he_kredi") <(echo "$he_age") --delimiters "\t")

In [ ]:
save data in a csv file , proper result

In [44]:
echo "$selected_info" | column -s $'\t' -t | head -10

10005103  690000    İkinci El         Uygun        20
1108042   440000    İkinci El         Uygun        24
12497729  490000    Sıfır             Uygun        3
12748522  580000    İkinci El         Uygun        38
14363743  645000    İkinci El         Uygun        20
1514239   420000    NA                Uygun        25
15395342  160000    İkinci El         Uygun değil  20
16603754  382000    İkinci El         Uygun        10
16723985  360000    Sıfır             Uygun        0
17462287  450000    İkinci El         Uygun değil  15


In [1]:
selected_info <- read.delim("selected_info.tsv", header = F)

In [2]:
str(selected_info)

'data.frame':	994 obs. of  5 variables:
 $ V1: int  10005103 1108042 12497729 12748522 14363743 1514239 15395342 16603754 16723985 17462287 ...
 $ V2: int  690000 440000 490000 580000 645000 420000 160000 382000 360000 450000 ...
 $ V3: Factor w/ 3 levels "S\304\261f\304\261r",..: 3 3 1 3 3 NA 3 3 1 3 ...
 $ V4: Factor w/ 3 levels "Bilinmiyor","Uygun",..: 2 2 2 2 2 2 3 2 2 3 ...
 $ V5: int  20 24 3 38 20 25 20 10 0 15 ...


In [3]:
selected_info_na <- na.omit(selected_info)

In [4]:
selected_info_na 

,V1,V2,V3,V4,V5
1,10005103,690000,kinci El,Uygun,20
2,1108042,440000,kinci El,Uygun,24
3,12497729,490000,Sfr,Uygun,3
4,12748522,580000,kinci El,Uygun,38
5,14363743,645000,kinci El,Uygun,20
7,15395342,160000,kinci El,Uygun de<9f>il,20
8,16603754,382000,kinci El,Uygun,10
9,16723985,360000,Sfr,Uygun,0
10,17462287,450000,kinci El,Uygun de<9f>il,15
11,17697337,480000,kinci El,Uygun,0


In [6]:
names(selected_info_na) <- c("id", "price", "situation", "eligibility", "age")

In [7]:
str((selected_info_na))

'data.frame':	905 obs. of  5 variables:
 $ id         : int  10005103 1108042 12497729 12748522 14363743 15395342 16603754 16723985 17462287 17697337 ...
 $ price      : int  690000 440000 490000 580000 645000 160000 382000 360000 450000 480000 ...
 $ situation  : Factor w/ 3 levels "S\304\261f\304\261r",..: 3 3 1 3 3 3 3 1 3 3 ...
 $ eligibility: Factor w/ 3 levels "Bilinmiyor","Uygun",..: 2 2 2 2 2 3 2 2 3 2 ...
 $ age        : int  20 24 3 38 20 20 10 0 15 0 ...
 - attr(*, "na.action")= 'omit' Named int  6 28 124 127 128 131 144 176 199 208 ...
  ..- attr(*, "names")= chr  "6" "28" "124" "127" ...


In [10]:
with(selected_info_na, aggregate(price, by = list(eligibility,situation), FUN = sum),order=price)

Group.1,Group.2,x
Bilinmiyor,Sfr,320000
Uygun,Sfr,162398500
Uygun de<9f>il,Sfr,450000
Uygun,Yapm A<9f>amasnda,630000
Uygun de<9f>il,Yapm A<9f>amasnda,295000
Bilinmiyor,kinci El,3950000
Uygun,kinci El,288213500
Uygun de<9f>il,kinci El,37989500
